# Winequality dataset

## Feim els imports

In [7]:
#IMPORTS
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

SyntaxError: invalid syntax (Temp/ipykernel_8136/3465534878.py, line 10)

## Preparam el dataset

Funció que separa el dataset en dues variables

In [ ]:
def load_data(df):
    y=df.quality
    X=df.drop('quality',1)
    X = min_max_scaler.fit_transform(X)
    return X, y

In [ ]:
#CREATE AND MERGE DATAFRAMES
dfRed=pd.read_csv("winequality-red.csv", sep=";")
dfWhite=pd.read_csv("winequality-white.csv", sep=";")

dfRed['Type']="Red"
dfWhite['Type']="White"
df=pd.concat([dfRed,dfWhite])

In [ ]:
df.dropna(inplace=True)

In [ ]:
labelencoder = LabelEncoder()
min_max_scaler = preprocessing.MinMaxScaler()

df['Type']=labelencoder.fit_transform(df['Type'])
df = df.sample(frac=1)

X, y = load_data(df)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
df.info()

In [ ]:
#LOGISTIC REGRESSION

logreg = LogisticRegression(max_iter=5000)
logreg.fit(X_train,y_train)

y_predict = logreg.predict(X_test)

print(classification_report(y_test, y_predict))

training_accuracy = logreg.score(X_train,y_train)
print('training_accuracy:',training_accuracy)

testing_accuracy = logreg.score(X_test,y_test)
print('testing_accuracy:',testing_accuracy)

In [ ]:
#PERCEPTRON
perceptron = Perceptron()
perceptron.fit(X_train, y_train)

y_predict=perceptron.predict(X_test)

print(classification_report(y_test, y_predict))

training_accuracy = perceptron.score(X_train,y_train)
print('training_accuracy:',training_accuracy)

testing_accuracy = perceptron.score(X_test,y_test)
print('testing_accuracy:',testing_accuracy)

In [ ]:
#RANDOM FOREST
rf = RandomForestClassifier(max_depth=14)
rf.fit(X_train, y_train)

y_predict=rf.predict(X_test)

print(classification_report(y_test, y_predict))

training_accuracy = rf.score(X_train,y_train)
print('training_accuracy:',training_accuracy)

testing_accuracy = rf.score(X_test,y_test)
print('testing_accuracy:',testing_accuracy)